In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder,OneHotEncoder,OrdinalEncoder
from pandas_profiling import ProfileReport
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [5]:
train = pd.read_csv('train.csv',index_col='id')
test = pd.read_csv('test.csv',index_col='id')

In [13]:
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
2,B,B,A,A,B,D,A,F,A,O,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
3,A,A,A,C,B,D,A,D,A,F,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
4,B,B,A,C,B,D,A,E,C,K,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
6,A,A,A,C,B,D,A,E,A,N,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [8]:
report = ProfileReport(train)

Summarize dataset:   0%|          | 0/39 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

TypeError: 'str' object is not callable

In [12]:
report.to_file('report.html')

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
cat_features = train.columns[0:10]

le = LabelEncoder()
ordinal_encoder = OrdinalEncoder()
train[cat_features] = ordinal_encoder.fit_transform(train[cat_features])


In [21]:
test[cat_features] = ordinal_encoder.transform(test[cat_features])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('target',axis=1),train['target'],random_state=0)

In [28]:
lgbm_parameters = {
    'metric': 'rmse',
    'n_jobs': -1,
    'n_estimators': 50000,
    'reg_alpha': 10.924491968127692,
    'reg_lambda': 17.396730654687218,
    'colsample_bytree': 0.21497646795452627,
    'subsample': 0.7582562557431147,
    'learning_rate': 0.009985133666265425,
    'max_depth': 18,
    'num_leaves': 63,
    'min_child_samples': 27,
    'max_bin': 523,
    'cat_l2': 0.025083670064082797
}

In [38]:
X = train.drop('target',axis=1)
y = train['target']

lgbm_val_pred = np.zeros(len(y))
lgbm_test_pred = np.zeros(len(test))
mse = []
kf = KFold(n_splits=10, shuffle=True)

for trn_idx, val_idx in tqdm(kf.split(X,y)):
    x_train_idx = X.iloc[trn_idx]
    y_train_idx = y.iloc[trn_idx]
    x_valid_idx = X.iloc[val_idx]
    y_valid_idx = y.iloc[val_idx]

    lgbm_model = LGBMRegressor(**lgbm_parameters)
    lgbm_model.fit(x_train_idx, y_train_idx,
                   eval_set = [(x_train_idx,y_train_idx),(x_valid_idx, y_valid_idx)],
                   verbose = -1, early_stopping_rounds = 400
                   #,categorical_feature=cat_features
                    )
    lgbm_test_pred += lgbm_model.predict(test)/10
    mse.append(mean_squared_error(y_valid_idx, lgbm_model.predict(x_valid_idx)))

np.mean(mse)
pd.DataFrame({'id':test.index,'target':lgbm_test_pred}).to_csv('submission.csv', index=False)

10it [23:10, 139.02s/it]


Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[3861]	training's rmse: 0.691336	valid_1's rmse: 0.717291
Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[3737]	training's rmse: 0.691819	valid_1's rmse: 0.718843
Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[3689]	training's rmse: 0.692682	valid_1's rmse: 0.713776
Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[4981]	training's rmse: 0.686659	valid_1's rmse: 0.70768
Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[4226]	training's rmse: 0.689579	valid_1's rmse: 0.716008
Training until validation scores don't improve for 400 rounds
Early stopping, best iteration is:
[3575]	training's rmse: 0.693066	valid_1's rmse: 0.717965
Training until validation scores don't improve for 400 rounds
Early sto

In [37]:
lgbm_model = LGBMRegressor(**lgbm_parameters)
lgbm_model.fit(X_train, y_train,
                   #eval_set = [(x_train_idx,y_train_idx),(x_valid_idx, y_valid_idx)],
                   verbose = -1, early_stopping_rounds = 400)

ValueError: For early stopping, at least one dataset and eval metric is required for evaluation